<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/Super_Piano_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Super Piano 2

Based on Yuankui Lee's repo and code https://github.com/djosix/Performance-RNN-PyTorch

MAKE YOUR OWN SOTA PIANO MUSIC AI MODEL IN UNDER 4 HOURS !!! :)

MAESTRO Dataset is courtesy of Google Magenta Team and it is distributed under Attribution-NonCommercial-ShareAlike 4.0 International license.

So keep this in mind and respect everyone's copyright, please :)

Huge thanks go out to all people who shared these amazing code contributions and made this Colab notebook possible :) Thank you so much, guys!

### Setup the environment

In [ ]:
#@title Install all dependencies and requrements
print(3..2..1..lets do it)
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install numpy
!pip install tensorboardX
!pip install tensorboard
!pip install progress
!pip install pretty-midi
!pip install pypianoroll
!pip install matplotlib
!pip install mir_eval
!pip install librosa
!git clone https://github.com/asigalov61/Performance-RNN-PyTorch
!nvidia-smi
print('Success :) Everything is installed and should work fine :) Enjoy!')

### Download and Unzip training MIDIs DataSet

In [ ]:
#@title (Best Choice/Works best stand-alone) Alex Piano Only Drafts Original 1500 MIDIs 
%cd /content/Performance-RNN-PyTorch/dataset/midi
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'

In [ ]:
#@title (Optional) Google Magenta MAESTRO Piano MIDI Dataset (~1300 MIDIs)
%cd /content/Performance-RNN-PyTorch/dataset/midi
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip

### Prepare and pre-process your MIDI DataSet for training

In [ ]:
#@title This may take a while, especially on the large DataSets, so please be patient
%cd /content/Performance-RNN-PyTorch
!python3 preprocess.py '/content/Performance-RNN-PyTorch/dataset/midi' '/content/Performance-RNN-PyTorch/dataset/processed' 64

### (Optional) Activate Tensorboard to monitor the progress of the model during training. You can also activate this cell at any other time to view logs/records of all training runs

In [ ]:
#@title Tensorboard Graphs and Stats
# Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime, os
%tensorboard --logdir /content/Performance-RNN-PyTorch/runs

###Train your model quickly here :) 

WARNING: Created/resulting Model may produce (partially) plagiarized (overfitted) output. Excercise care and respect the copyright, please :) NOTE: You can manipulate provided variables below to further influence/improve generated output. Only the first batch is downloaded and plotted.

In [ ]:
#@title Main Training Loop
save_model_every = 200 #@param {type:"slider", min:0, max:1000, step:10}
number_of_batches = 256 #@param {type:"slider", min:0, max:512, step:1}
window_size = 512 #@param {type:"slider", min:0, max:512, step:1}
%cd /content/Performance-RNN-PyTorch
!python3 train.py -s save/myModel.sess -d '/content/Performance-RNN-PyTorch/dataset/processed' -i 200 -b 256 -w 512 -L

###Generate, Plot, Graph, Save, Download, and Render the resulting output

In [ ]:
#@title Main Generation Loop
number_of_tokens_to_generate =  1024#@param {type:"integer"}
model_temperature = 0.7 #@param {type:"slider", min:0, max:3, step:0.1}
number_of_batches_and_files_to_generate = 1 #@param {type:"slider", min:0, max:16, step:1}


!python3 generate.py -l 1024 -T 0.7 -b 4 -s '/content/Performance-RNN-PyTorch/save/myModel.sess'
print('Successfully exported the output to output-000.mid')
print('Downloading output-000.mid')
from google.colab import files
files.download('/content/Performance-RNN-PyTorch/output/output-000.mid')

In [ ]:
#@title Plot/Graph the Output and Render it for listening (SineWaves)
graphs_length_inches = 18 #@param {type:"slider", min:0, max:20, step:1}
notes_graph_height = 4 #@param {type:"slider", min:0, max:20, step:1}
rendered_wav_graph_height = 3 #@param {type:"slider", min:0, max:20, step:1}
import librosa
import numpy as np
import pretty_midi
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('SVG')
# For plotting
import mir_eval.display
import librosa.display
%matplotlib inline


midi_data = pretty_midi.PrettyMIDI('/content/Performance-RNN-PyTorch/output/output-000.mid')

def plot_piano_roll(pm, start_pitch, end_pitch, fs=100):
    # Use librosa's specshow function for displaying the piano roll
    librosa.display.specshow(pm.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))



roll = np.zeros([int(graphs_length_inches), 128])
# Plot the output

track = Multitrack('/content/Performance-RNN-PyTorch/output/output-000.mid', name='track')
plt.figure(figsize=[graphs_length_inches, notes_graph_height])
fig, ax = track.plot()
fig.set_size_inches(graphs_length_inches, notes_graph_height)
plt.figure(figsize=[graphs_length_inches, notes_graph_height])
ax2 = plot_piano_roll(midi_data, 24, 84)
plt.show(block=False)

# Generate rendering (WAV)


audio = midi_data.synthesize()

print(audio.shape)

plt.figure(figsize=[graphs_length_inches, rendered_wav_graph_height])
plt.plot(audio)
plt.show(block=False)

import IPython.display as ipd
ipd.Audio(audio, rate=16000)